# Introduction to Community Modeling

Author: Daniel Machado, NTNU

License: [CC BY-SA 4.0](http://creativecommons.org/licenses/by-sa/4.0/)

-------

In this tutorial:

- You will learn how to perform flux balance analysis of microbial communities
using a model of the [central carbon metabolism of *E. coli*](https://journals.asm.org/doi/10.1128/ecosalplus.10.2.1).

- You will use the [ReFramed](https://github.com/cdanielmachado/reframed) python library for metabolic modeling. 

## Step 1: Setting up a community

We will create a synthetic microbial consortium with two *E. coli* mutants growing in minimal medium. In one of the mutants we will knockout the glucose transporter and in the other we will knockout the ammonium transporter.

![synthetic community](../files/synthetic_community.png)

As usual, we start by loading the model for the wild-type:

In [ ]:
from reframed import load_cbmodel
wildtype = load_cbmodel('../files/e_coli_core.xml')

Now we create our two mutants (`glc_ko` and `nh4_ko`):

In [ ]:
glc_ko = wildtype.copy()
glc_ko.id = 'glc_ko'
glc_ko.set_flux_bounds('R_GLCpts', 0, 0)

nh4_ko = wildtype.copy()
nh4_ko.id = 'nh4_ko'
nh4_ko.set_flux_bounds('R_NH4t', 0, 0)

**ReFramed** has some basic functionality for working with microbial communities, one is the `Community` class to create microbial communities from a list of models of individual species: 

In [ ]:
from reframed import Community
community = Community('ecoli_pair', [glc_ko, nh4_ko])

This community model ignores the environmental conditions that were specified in the original models (since these could be very different). 

To make our life easier, we will extract the nutrient composition specified in the wild-type model to use later.

In [ ]:
from reframed import Environment

M9 = Environment.from_model(wildtype)
print(f"Environment compounds: {', '.join(M9.get_compounds())}")

## Step 2: Simulation using (conventional) FBA

A very simple way to simulate a microbial community is to merge the individual models into a single model that mimics a "super organism", where each microbe lives inside its own compartment, and run a (conventional) FBA simulation for this *super organism*.

In [ ]:
from reframed import FBA

super_oganism = community.merged_model
solution = FBA(super_oganism, constraints=M9)

print(solution)
solution.show_values(pattern='R_EX')

We can see that the model predicts a growth rate (total biomass per hour) similar to the wild-type, with an efficient consumption of glucose and ammonia that results in respiratory metabolism.

But what is each organism doing, and are both organisms actually growing at the same rate?

Let's print the (non-zero) fluxes for each organism:

In [ ]:
solution.show_values(pattern='_glc_ko')

In [ ]:
solution.show_values(pattern='_nh4_ko')

Actually it seems that only one of the organisms is growing while the other has an active metabolism (it exchanges metabolites with the environment and with the other organism) performing the role of a bioconverter, but none of the flux is used for growth. 

> Do you think this would be a stable consortium ?

## Step 3: Community Simulation with SteadyCom

**SteadyCom** by [Chan, et al (2017)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005539) is a recent community simulation method that takes into account the fact that to reach a stable composition the organisms need to grow at the same *specific growth rate* (1/h), which means that the *absolute growth rate* (gDW/h) of each organism is proportional to its *abundance* at steady-state (gDW).

Let's simulate the same community using SteadyCom:

In [ ]:
from reframed import SteadyCom
solution = SteadyCom(community, constraints=M9)

In this case the solution object shows the overall community growth rate and the relative abundance of each species:

In [ ]:
print(solution)

The `solution` object for community simulations implements a few additional features, such as enumerating all the cross-feeding interactions:

In [ ]:
solution.cross_feeding(as_df=True).fillna('environment').sort_values(['donor', 'receiver'])

We can plot the fluxes of each mutant in a map to help with interpretation of the results:

In [ ]:
from reframed import fluxes2escher
fluxes2escher(solution.internal['glc_ko'])

In [ ]:
fluxes2escher(solution.internal['nh4_ko'])

## Step 4: Explore alternative solutions

> Look more closely at the compounds that are exchanged between the two mutants and also at their relative abundance. Is this what you expected? Do you think there could be different solutions?

Unfortunately, one limitation of **SteadyCom**, which is exemplified by [Chan, et al (2017)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005539) in Figure 3 (reproduced below), is the variability in the solution space when the community is not growing at the maximum (theoretical) growth rate.

![variability](../files/steadycom_variability.png)

> Would you expect a synthetic community to grow at its maximum growth rate?

**ReFramed** implements a variability analysis function for the SteadyCom solution space, let's see what happens if the community is growing at 90% of the theoretical maximum:

In [ ]:
from reframed import SteadyComVA
variability = SteadyComVA(community, obj_frac=0.95, constraints=M9)

print('Strain\tMin\tMax')
for strain, (lower, upper) in variability.items():
    print(f'{strain}\t{lower:.1%}\t{upper:.1%}')

As you can see, there is a really large variability in this solution space. This means that we know in theory the two mutants **can** cooperate and survive in minimal media, but there is still a lot of uncertainty with regard to **how** they will achieve a stable consortium.

> How do you think we can reduce this uncertainty?

In [ ]:
# Feel free to play around with these examples.
# Type your own code here...